<a href="https://colab.research.google.com/github/Eduarmma19/videos_youtube/blob/main/Extraccion_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
#Importación de librerías necesarias
library(dplyr)
library(rvest)

In [66]:
#Url de la pagina 
url <- "https://computacion.mercadolibre.com.co/accesorios-pc-gaming-audifonos/diadema-gamer"
pagina <- read_html(url)


In [67]:
#Extraer el nombre de cada producto
nombres <- pagina %>% html_nodes(".ui-search-item__title")%>% html_text()

In [68]:
#Extraer el precio de cada producto
precios <- pagina %>% html_nodes(".ui-search-price--size-medium .ui-search-price__second-line .price-tag-fraction") %>% html_text

In [69]:
#Crear data frame con el nombre y precios de los productos
df <- data.frame(nombres,precios)

In [70]:
#Mostrar los primeros registros del data frame (df)
head(df)

,nombres,precios
,<chr>,<chr>
1,Audifonos Diadema Gamer Usb Para Xboxone Ps4 Pc Luz Led,70.900
2,"Diadema Gamer Auricular Con Microfono Ps4, Pc, Xbox, Swich",70.000
3,Audifono Diadema Cascos Gamer Pro2 Para Ps4 Y Xbox One,70.000
4,Audifonos Diadema Gamer Kotion G2000 Con Microfono,58.800
5,Audifono Diadema Cascos Gamer Pro2 Para Ps4 Y Xbox One,70.000
6,Audifonos Gamer En Diadema Kotion G2000 Microfono Usb Y Led,59.900


In [71]:
#Obtener los links que me llevan a cada producto de la primera pagina de busqueda (href)
links_productos <- pagina %>% html_nodes(".ui-search-result__image > a") %>% html_attr("href")
length(links_productos)

[1] 56

In [72]:
#Crear función que permite ingresar a cada producto y tomar las ventas durante los últimos 60 días y la atención brindada por el proveedor 
#Como salida tendremos un data frame con la variable ventas y atención. 

obtener_info <- function(link){
  pagina_producto <- read_html(link)
  ventas <- pagina_producto %>%html_nodes(".ui-pdp-seller__sales-description")  %>% html_text()
  atencion <- pagina_producto %>% html_nodes(".ui-pdp-seller__item-description:nth-child(2) .ui-pdp-seller__text-description")%>% html_text()
  df <- data.frame(ventas,atencion)
  df   
  }

In [73]:
#Aplicación de la función creada a cada uno de los links que nos lleva a los productos
df_total <- sapply(links_productos,obtener_info,USE.NAMES = FALSE)


In [ ]:
head(df_total)

ventas,658,10772,10772,3970,9710,1820,10560,99,129,9710,⋯,610,2873,3194,1820,2264,851,2873,1363,7319,3970
atencion,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención,⋯,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención,Brinda buena atención


In [74]:
#Transformaciones necesarias para el df_final 
df_total <- t(df_total)

In [75]:
#Crear el data frame final (df_final )
df_final <- cbind(df,df_total)

In [76]:
head(df_final)

,nombres,precios,ventas,atencion
,<chr>,<chr>,<list>,<list>
1,Audifonos Diadema Gamer Usb Para Xboxone Ps4 Pc Luz Led,70.900,1921,Brinda buena atención
2,"Diadema Gamer Auricular Con Microfono Ps4, Pc, Xbox, Swich",70.000,403,Brinda buena atención
3,Audifono Diadema Cascos Gamer Pro2 Para Ps4 Y Xbox One,70.000,10600,Brinda buena atención
4,Audifonos Diadema Gamer Kotion G2000 Con Microfono,58.800,10600,Brinda buena atención
5,Audifono Diadema Cascos Gamer Pro2 Para Ps4 Y Xbox One,70.000,3924,Brinda buena atención
6,Audifonos Gamer En Diadema Kotion G2000 Microfono Usb Y Led,59.900,9590,Brinda buena atención


In [78]:
#Tenemos los resultados de la primera página de busqueda, ahora vamos a extraer los datos de la 2da y 3ra página de busqueda
#Que tienen en común los demás enlaces que corresponden a la 2da y 3ra página

# https://computacion.mercadolibre.com.co/accesorios-pc-gaming-audifonos/diadema-gamer_Desde_51
# https://computacion.mercadolibre.com.co/accesorios-pc-gaming-audifonos/diadema-gamer_Desde_101
# https://computacion.mercadolibre.com.co/accesorios-pc-gaming-audifonos/diadema-gamer_Desde_151
seq(from = 51,to = 151,by = 50)

[1]  51 101 151

In [79]:
#Crear un data frame vacio 
df_paginas <- data.frame()

In [80]:
#Extraer los datos de la 2da y 3ra página 
for(n_pagina in seq(from = 51,to = 151,by = 50)){
  url_paginas <- paste0("https://computacion.mercadolibre.com.co/accesorios-pc-gaming-audifonos/diadema-gamer_Desde_",n_pagina)
  pagina_n <- read_html(url_paginas)

  nombres <- pagina_n %>% html_nodes(".ui-search-item__title")%>% html_text()
  precios <- pagina_n %>% html_nodes(".ui-search-price--size-medium .ui-search-price__second-line .price-tag-fraction") %>% html_text
  links_productos <- pagina_n %>% html_nodes(".ui-search-result__image > a") %>% html_attr("href")
  df_n <- sapply(links_productos,obtener_info,USE.NAMES = FALSE)
  df_n <- t(df_n)

  df_n <- as.data.frame(df_n)

  ventas <- unlist(df_n$ventas)
  atencion <- unlist(df_n$atencion)

  df_paginas <- rbind(df_paginas,data.frame(nombres,precios,ventas,atencion))
}


In [82]:
df_paginas

nombres,precios,ventas,atencion
<chr>,<chr>,<chr>,<chr>
Audifonos Gamer Diadema G9000 Kotion Ps4 Xboxone + Obsequio,60.900,2159,Brinda buena atención
"Logitech G Pro X, Diadema Gamer 7.1 Profesional, Pc Ps4 Xbox",649.900,10458,Brinda buena atención
Diadema Audifono Gamer Rgb Light Usb Microfono,49.900,5657,Brinda buena atención
Diadema Arctis 5 Steelseries 7.1 Surround Rgb,399.000,528,Brinda buena atención
Diadema Gamer Redragon H120 Ares 3.5 Mm,53.900,411,Brinda buena atención
"Diadema Usb Micrófono Control Volumen, X-kim Hf-868u - Negro",59.900,130,Brinda buena atención
Diadema Auricular / Headset Gamer Scylla H901 - Redragon,98.900,888,Brinda buena atención
Diadema Gamer Hyperx Cloud Stinger Core 7.1 Inalambrica,319.000,3166,Brinda buena atención
"Diadema Usb Star Tec Hp-20u, Micrófono / Control De Volumen",62.900,12334,Brinda buena atención
